# Commit generating commits

In [1]:
#!pip install --upgrade bottleneck

import psycopg2
import pandas as pd 
import sqlalchemy as salc
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("../../config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

## List Available Repositories

In [2]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""SELECT a.rg_name,
                                      a.repo_group_id,
                                      b.repo_name,
                                      b.repo_id,
                                      b.forked_from,
                                      b.repo_archived 
                                      FROM repo_groups a, repo b 
                                      WHERE a.repo_group_id = b.repo_group_id 
                                      ORDER BY rg_name, repo_name;

                            """)
repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived
0,-,26301,test,257882,Parent not available,0.0
1,18f,25602,10x-apis-xtravaganza,125279,Parent not available,1.0
2,18f,25602,10x-dux-app,125423,Parent not available,1.0
3,18f,25602,10x-dux-vuls-eval,125425,Parent not available,1.0
4,18f,25602,10x-mel,125410,Parent not available,1.0
...,...,...,...,...,...,...
103861,zotero,25444,zotero-standalone-build,27218,Parent not available,0.0
103862,zotero,25444,zotero-word-for-mac-integration,27192,Parent not available,0.0
103863,zotero,25444,zotero-word-for-windows-integration,27198,Parent not available,0.0
103864,zotero,25444,None,27230,Parent not available,0.0


rg_name           object
repo_group_id      int64
repo_name         object
repo_id            int64
forked_from       object
repo_archived    float64
dtype: object

## Commits Data: Number of Commits


In [3]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
            select rg_name, repo_group_id, repo_name, d.repo_id, repo_git, forked_from, repo_archived, count(cmt_commit_hash) as commits
            from (
             SELECT a.rg_name as rg_name,
                a.repo_group_id as repo_group_id,
                b.repo_name as repo_name,
                b.repo_id as repo_id,
                b.repo_git as repo_git, 
                b.forked_from as forked_from,
                b.repo_archived as repo_archived
            FROM
                repo_groups a,
                repo b
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name) d, commits c 
            where d.repo_id = c.repo_id 
            group by rg_name, repo_git, repo_group_id, repo_name, d.repo_id, forked_from, repo_archived
                

    """)

#repo_query_str = str(repo_query)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,repo_git,forked_from,repo_archived,commits
0,18f,25602,10x-dux-app,125423,https://github.com/18f/10x-dux-app,Parent not available,1,84
1,18f,25602,10x-dux-vuls-eval,125425,https://github.com/18f/10x-dux-vuls-eval,Parent not available,1,205
2,18f,25602,10x-mel,125410,https://github.com/18f/10x-mel,Parent not available,1,139
3,18f,25602,10x-mlaas,125245,https://github.com/18f/10x-mlaas,Parent not available,1,307
4,18f,25602,10x-modern-contract-vehicles,125272,https://github.com/18f/10x-modern-contract-veh...,Parent not available,1,2
...,...,...,...,...,...,...,...,...
103197,zotero,25444,zotero-maps,27224,https://github.com/zotero/zotero-maps,schuyler/zotero-maps,0,63
103198,zotero,25444,zotero-schema,27431,https://github.com/zotero/zotero-schema,Parent not available,0,62
103199,zotero,25444,zotero-standalone-build,27218,https://github.com/zotero/zotero-standalone-build,Parent not available,0,1526
103200,zotero,25444,zotero-word-for-mac-integration,27192,https://github.com/zotero/zotero-word-for-mac-...,Parent not available,0,1256


rg_name          object
repo_group_id     int64
repo_name        object
repo_id           int64
repo_git         object
forked_from      object
repo_archived     int64
commits           int64
dtype: object

### Write Commits Data to a CSV

In [4]:
repolist.to_csv(path_or_buf='commits_with_git_url.csv')

### Number of Contributors/committers

In [5]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
        select rg_name, repo_group_id, repo_name, repo_id, repo_git, forked_from, repo_archived,  count(*) as committer_count from 
        (
            select rg_name, repo_group_id, repo_name, d.repo_id, repo_git, forked_from, repo_archived, cmt_ght_author_id, count(cmt_ght_author_id) as comitter_count
            from (
             SELECT a.rg_name as rg_name,
                a.repo_group_id as repo_group_id,
                b.repo_name as repo_name,
                b.repo_id as repo_id,
                b.repo_git as repo_git, 
                b.forked_from as forked_from,
                b.repo_archived as repo_archived
            FROM
                repo_groups a,
                repo b
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name) d, commits c 
            where d.repo_id = c.repo_id 
            group by rg_name, repo_git, repo_group_id, repo_name, d.repo_id, forked_from, repo_archived, cmt_ght_author_id
        ) f 
        group by rg_name, repo_group_id, repo_name, repo_id, repo_git, forked_from, repo_archived

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

In [ ]:
repolist.to_csv(path_or_buf='committer_count_by_repo.csv')

In [ ]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
            select repo_id, count(cmt_ght_author_id) as null_committers from augur_data.commits where cmt_ght_author_id is null 
            group by repo_id 
            order by null_committers desc; 
    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes



,repo_id,null_committers
0,1,0
1,25481,0
2,25483,0
3,25487,0
4,25488,0
...,...,...
27466,191515,0
27467,191516,0
27468,191517,0
27469,191520,0


repo_id            int64
null_committers    int64
dtype: object

### Committer Identity Information

In [ ]:
commit_count = pd.DataFrame()

commit_count_qry = salc.sql.text(f"""
        select cmt_author_email, cmt_commit_hash, count(*) as counter 
        from commits where repo_id = 1 
        group by cmt_author_email, cmt_commit_hash
        order by counter desc; 
        -- "commits" is a misnomer for the table name ... it contains commit_files
        -- This shows me that s@goggins.com has the two most giant commits ...  
    """)

commit_count = pd.read_sql(commit_count_qry, con=engine)

display(commit_count)

commit_count.dtypes

,cmt_author_email,cmt_commit_hash,counter
0,s@goggins.com,8aa5aee59c2371ee743b5dc2509e7fd37c828455,1853
1,s@goggins.com,b6b82695004218e1b44ac08f5aaad2b7063abfe9,1804
2,61482022+ABrain7710@users.noreply.github.com,cba1d9cb141909810f65dfc91e274a90526c2073,839
3,61482022+ABrain7710@users.noreply.github.com,a5b5e8e99459854944ea4b085f39490a6507861c,561
4,gabe.heim@yahoo.com,75c9a7a3ea8ae9c1c6f8f91b4385d657e48bf34f,430
...,...,...,...
11109,61482022+ABrain7710@users.noreply.github.com,f5ce0a5079c06c1ea0b2bcd3df84125ad78a62dc,1
11110,gabe.heim@yahoo.com,5986c3105c996cd950f04a01b870eaf1c64463a3,1
11111,s@goggins.com,bf686aee1e550cd990d2c600c0da3834701b4c0c,1
11112,krabs@tilde.team,40614b14162dabca7394112ac5884c312f58689c,1


cmt_author_email    object
cmt_commit_hash     object
counter              int64
dtype: object

In [ ]:
import pandas as pd
import sqlalchemy as salc

commit_time = pd.DataFrame()

commit_time_qry = salc.sql.text("""
        SELECT
            rg.rg_name,
            r.repo_group_id,
            r.repo_name,
            r.repo_id,
            r.repo_git,
            r.forked_from,
            r.repo_archived,
            COUNT(DISTINCT c.cmt_ght_author_id) as committer_count
        FROM
            repo_groups rg
        JOIN
            repo r ON rg.repo_group_id = r.repo_group_id
        JOIN
            commits c ON r.repo_id = c.repo_id
        WHERE
            c.cmt_author_date <> '' AND
            c.cmt_author_date IS NOT NULL AND
            CAST(c.cmt_author_date AS TIMESTAMP) BETWEEN NOW() - INTERVAL '18 months' AND NOW()
        GROUP BY
            rg.rg_name,
            r.repo_group_id,
            r.repo_name,
            r.repo_id,
            r.repo_git,
            r.forked_from,
            r.repo_archived
""")

commit_time = pd.read_sql(commit_time_qry, con=engine)

display(commit_time)

# Checking the data types of the resulting DataFrame
commit_time.dtypes


,rg_name,repo_group_id,repo_name,repo_id,repo_git,forked_from,repo_archived,committer_count
0,18f,25602,api.data.gov,124386,https://github.com/18f/api.data.gov,Parent not available,0,2
1,18f,25602,18f.gsa.gov,124400,https://github.com/18f/18f.gsa.gov,Parent not available,0,44
2,18f,25602,charlie,124457,https://github.com/18f/charlie,Parent not available,0,11
3,18f,25602,analytics-reporter,124486,https://github.com/18f/analytics-reporter,Parent not available,0,7
4,18f,25602,analytics.usa.gov,124499,https://github.com/18f/analytics.usa.gov,Parent not available,0,8
...,...,...,...,...,...,...,...,...
40921,zotero,25444,note-editor,27457,https://github.com/zotero/note-editor,Parent not available,0,2
40922,zotero,25444,pdf.js,27463,https://github.com/zotero/pdf.js,mozilla/pdf.js,0,19
40923,zotero,25444,utilities,27468,https://github.com/zotero/utilities,Parent not available,0,4
40924,zotero,25444,markdown-translator-builder,27472,https://github.com/zotero/markdown-translator-...,false,0,1


rg_name            object
repo_group_id       int64
repo_name          object
repo_id             int64
repo_git           object
forked_from        object
repo_archived       int64
committer_count     int64
dtype: object

In [ ]:
commit_time.to_csv(path_or_buf='committer_time.csv')